In [1]:
import atexit
import gym
import json
from kaggle_environments import make
import math
from pathlib import Path
import random
from scipy.stats import rankdata
from subprocess import Popen, PIPE
from typing import *

from lux_ai.lux.game import Game
from lux_ai.lux_env import LuxEnv

DIR_PATH = Path("/home/isaiah/GitHub/Kaggle/Lux_AI_2021/lux_ai/")
dimension_process = Popen(
    ["node", str(DIR_PATH / "dimensions/main.js")],
    stdin=PIPE,
    stdout=PIPE
)

env = make("lux_ai_2021")
env.reset()

initiate = {
    "type": "start",
    "agent_names": [], # unsure if this is provided?
    "config": env.configuration
}
dimension_process.stdin.write((json.dumps(initiate) + "\n").encode())
dimension_process.stdin.flush()

agent1res = json.loads(dimension_process.stdout.readline())
agent2res = json.loads(dimension_process.stdout.readline())

game_state = Game()
game_state._initialize(agent1res)
game_state._update(agent1res)

Loading environment football failed: No module named 'gfootball'


In [2]:
env = LuxEnv()
env.reset()

(<lux_ai.lux.game.Game at 0x7f39e8317160>, [0.0, 0.0], False, {})

{}

In [39]:
env.reset()
while not env.done:
    if (env.game_state.turn // 2) % 2 == 0:
        move = 'n'
    else:
        move = 's'
    # print(move)
    if len(env.game_state.players[0].units) > 0:
        actions = [[env.game_state.players[0].units[0].move(move)], []]
    else:
        actions = [[], []]
    returned = env.step(actions)

In [40]:
env.game_state.players[1].cities, env.game_state.players[0].units, returned

({}, [], (<lux_ai.lux.game.Game at 0x7f399f349640>, [0.0, 0.0], True, {}))

In [25]:
state = [{
    'action': [p.units[0].move('n'), list(p.cities.values())[0].citytiles[0].research()],
    #'reward': 0,
    #'info': {},
    #'observation': {},
    #'status': {}
} for p in game_state.players] * 2
print(state)
dimension_process.stdin.write((json.dumps(state) + "\n").encode())
dimension_process.stdin.flush()

[{'action': ['m u_1 n', 'r 5 7']}, {'action': ['m u_2 n', 'r 10 7']}, {'action': ['m u_1 n', 'r 5 7']}, {'action': ['m u_2 n', 'r 10 7']}]


In [24]:
[l for l in json.loads(dimension_process.stdout.readline()) if l.startswith('u') or l.startswith('rp')]

[]

In [28]:
json.loads(dimension_process.stdout.readline())

{'status': 'running', 'turn': 1, 'max': 361}

'm u_1 NORTH'

In [43]:
dimension_process.stdin.write((json.dumps(env.state) + "\n").encode())

2856

In [51]:
dimension_process.stdout.readline()

b'["rp 0 0","rp 1 0","r uranium 0 2 270","r coal 0 4 336","r coal 1 3 426","r coal 1 4 369","r coal 2 11 312","r wood 3 2 867","r wood 3 5 1164","r wood 4 0 869","r wood 4 2 1180","r wood 4 3 1204","r wood 4 5 1077","r wood 4 6 1026","r uranium 5 11 252","r coal 7 0 440","r wood 7 5 850","r wood 7 11 951","r wood 8 5 770","r wood 8 11 1122","r wood 9 0 866","r wood 9 1 1050","r wood 9 5 850","r wood 9 6 1221","r wood 9 7 1030","r wood 10 0 1260","r wood 10 6 1140","r wood 11 0 1261","r coal 11 9 360","r wood 11 10 1073","r wood 12 0 1261","r coal 12 9 360","r wood 12 10 1073","r wood 13 0 1260","r wood 13 6 1140","r wood 14 0 866","r wood 14 1 1050","r wood 14 5 850","r wood 14 6 1221","r wood 14 7 1030","r wood 15 5 770","r wood 15 11 1122","r coal 16 0 440","r wood 16 5 850","r wood 16 11 951","r uranium 18 11 252","r wood 19 0 869","r wood 19 2 1180","r wood 19 3 1204","r wood 19 5 1077","r wood 19 6 1026","r wood 20 2 867","r wood 20 5 1164","r coal 21 11 312","r coal 22 3 426","r 

In [18]:
game_state._update(agent1res)

In [41]:
env.step([['f'], ['f']])

[{'action': ['f'],
  'reward': 1001,
  'info': {},
  'observation': {'remainingOverageTime': 60,
   'step': 3,
   'reward': 1001,
   'player': 0,
   'updates': ['rp 0 0',
    'rp 1 0',
    'r uranium 0 2 270',
    'r coal 0 4 336',
    'r coal 1 3 426',
    'r coal 1 4 369',
    'r coal 2 11 312',
    'r wood 3 2 867',
    'r wood 3 5 1164',
    'r wood 4 0 869',
    'r wood 4 2 1180',
    'r wood 4 3 1204',
    'r wood 4 5 1077',
    'r wood 4 6 1026',
    'r uranium 5 11 252',
    'r coal 7 0 440',
    'r wood 7 5 850',
    'r wood 7 11 951',
    'r wood 8 5 790',
    'r wood 8 11 1122',
    'r wood 9 0 866',
    'r wood 9 1 1050',
    'r wood 9 5 850',
    'r wood 9 6 1221',
    'r wood 9 7 1030',
    'r wood 10 0 1260',
    'r wood 10 6 1140',
    'r wood 11 0 1261',
    'r coal 11 9 360',
    'r wood 11 10 1073',
    'r wood 12 0 1261',
    'r coal 12 9 360',
    'r wood 12 10 1073',
    'r wood 13 0 1260',
    'r wood 13 6 1140',
    'r wood 14 0 866',
    'r wood 14 1 1050',
   